# **Create Tables Data**

### Imports & Functions

In [1]:
import numpy as np
import pandas as pd
import re
from tqdm.notebook import tqdm

In [2]:
def standings(frame, result_col, goals_col, goals_opp_col, points_col):
    """This function takes in a DataFrame and strings identifying fields
    to calculate the league table.
    
    Making it generalized will allow us to calculate league tables for
    First Half Goals only. Second Half Goals only.
    """
    record = {}
    record['Played'] = np.size(frame[result_col])
    record['Won'] = np.sum(frame[result_col] == 'W')
    record['Drawn'] = np.sum(frame[result_col] == 'D')
    record['Lost'] = np.sum(frame[result_col] == 'L')
    record['GF'] = np.sum(frame[goals_col])
    record['GA'] = np.sum(frame[goals_opp_col])
    record['GD'] = record['GF'] - record['GA']
    record['Points'] = np.sum(frame[points_col])
    
    return pd.Series(record,index=['Played', 'Won', 'Drawn', 'Lost', 'GF', 'GA', 'GD', "Points"])

In [3]:
def get_result(score, score_opp):
    if score == score_opp:
        return 'D'
    elif score > score_opp:
        return 'W'
    else:
        return 'L'

## Get Tables Complete Data

In [5]:
results = pd.read_csv('input_tables.csv')
results.rename(columns={'league_id':'Div','time_starting_at_date_time':'Date','scores_home_score':'FTHG','scores_away_score':'FTAG','result':'FTR', 'home_name':'HomeTeam', 'away_name':'AwayTeam'}, inplace=True)
results['Date'] = pd.to_datetime(results['Date'], format = '%Y-%m-%d').dt.date
results['FTR'] = np.where(results['FTR'] == 0, 'D', np.where(results['FTR'] == 1, 'H', 'A'))
results['HTHG'] = [int(re.search('(\d+)-(\d+)', values).group(1)) for values in results['scores_ht_score']]
results['HTAG'] = [int(re.search('(\d+)-(\d+)', values).group(2)) for values in results['scores_ht_score']]
results['HTR'] = np.where(results['HTHG'] > results['HTAG'], 'H', np.where(results['HTHG'] == results['HTAG'], 'D', 'A'))
results.drop(['league_name', 'season_name', 'scores_ht_score'], axis=1, inplace=True)
results['H'] = results['HomeTeam']
results['A'] = results['AwayTeam']
cols_to_keep = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'id', 'round_name', 'season_id']
appended_data = []

for i in tqdm(results['season_id'].unique()):
    for j in results['round_name'].unique():
        resultG = results[(results['season_id'] == i) & (results['round_name'] <= j)]
        team_results = pd.melt(resultG, id_vars=cols_to_keep, value_vars=['H', 'A'], var_name='Home/Away', value_name='Team')
        team_results['Opponent'] = np.where(team_results['Team'] == team_results['HomeTeam'], team_results['AwayTeam'], team_results['HomeTeam'])
        points_map = {'W': 3,'D': 1,'L': 0}
        # full time goals
        team_results['Goals'] = np.where(team_results['Team'] == team_results['HomeTeam'],team_results['FTHG'],team_results['FTAG'])
        team_results['Goals_Opp'] = np.where(team_results['Team'] != team_results['HomeTeam'],team_results['FTHG'],team_results['FTAG'])
        team_results['Result'] = np.vectorize(get_result)(team_results['Goals'], team_results['Goals_Opp'])
        team_results['Points'] = team_results['Result'].map(points_map)
        # 1st half goals
        team_results['1H_Goals'] = np.where(team_results['Team'] == team_results['HomeTeam'],team_results['HTHG'],team_results['HTAG'])
        team_results['1H_Goals_Opp'] = np.where(team_results['Team'] != team_results['HomeTeam'],team_results['HTHG'],team_results['HTAG'])
        team_results['1H_Result'] = np.vectorize(get_result)(team_results['1H_Goals'], team_results['1H_Goals_Opp'])
        team_results['1H_Points'] = team_results['1H_Result'].map(points_map)
        # 2nd half goals
        team_results['2H_Goals'] = team_results['Goals'] - team_results['1H_Goals']
        team_results['2H_Goals_Opp'] = team_results['Goals_Opp'] - team_results['1H_Goals_Opp']
        team_results['2H_Result'] = np.vectorize(get_result)(team_results['2H_Goals'], team_results['2H_Goals_Opp'])
        team_results['2H_Points'] = team_results['2H_Result'].map(points_map)
        # Drop unnecessary columns and sort by date
        cols_to_drop = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
        team_results = (team_results.sort_values(by=['Date', 'id']))
        # Get League Table
        results_byteam = team_results.groupby(['Team'])
        # Rank Teams in Standings
        table_FT = (results_byteam.apply(standings,result_col='Result',goals_col='Goals',goals_opp_col='Goals_Opp',points_col='Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False))
        table_FT['rank'] = (table_FT.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int))
        table_FT['Type'] = 'FT'
        # Get League Table for First Half Goals only
        table_1H = (results_byteam.apply(standings,result_col='1H_Result',goals_col='1H_Goals',goals_opp_col='1H_Goals_Opp',points_col='1H_Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False))
        table_1H['rank'] = (table_1H.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int))
        table_1H['Type'] = '1H'
        # Get League Table_2H for First Half Goals only
        table_2H = (results_byteam.apply(standings,result_col='2H_Result',goals_col='2H_Goals',goals_opp_col='2H_Goals_Opp',points_col='2H_Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False))
        table_2H['rank'] = (table_2H.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int))
        table_2H['Type'] = '2H'
        table_merged = pd.concat([table_FT,table_1H, table_2H])
        table_merged['round_name'] = j
        table_merged['season_id'] = i
        appended_data.append(table_merged)
# see pd.concat documentation for more info
appended_data = pd.concat(appended_data)

  0%|          | 0/30 [00:00<?, ?it/s]

#### Checks and Export

In [6]:
print(len(appended_data))
appended_data.head(300)

67026


,Played,Won,Drawn,Lost,GF,GA,GD,Points,rank,Type,round_name,season_id
Team,,,,,,,,,,,,
Olympique Lyonnais,1,1,0,0,3,0,3,3,1,FT,1,765
Bordeaux,1,1,0,0,3,2,1,3,2,FT,1,765
Caen,1,1,0,0,3,2,1,3,2,FT,1,765
Metz,1,1,0,0,3,2,1,3,2,FT,1,765
Montpellier,1,1,0,0,1,0,1,3,5,FT,1,765
...,...,...,...,...,...,...,...,...,...,...,...,...
Lille,5,1,1,3,3,6,-3,4,16,2H,5,765
Bastia,5,1,1,3,2,5,-3,4,17,2H,5,765
Nancy,5,1,1,3,2,5,-3,4,17,2H,5,765


In [7]:
appended_data.to_csv('Output/tables_FULL.csv')

## Get Tables Home/Away Data

In [7]:
results = pd.read_csv('input_tables.csv')
results.rename(columns={'league_id':'Div','time_starting_at_date_time':'Date','scores_home_score':'FTHG','scores_away_score':'FTAG','result':'FTR', 'home_name':'HomeTeam', 'away_name':'AwayTeam'}, inplace=True)
results['Date'] = pd.to_datetime(results['Date'], format = '%Y-%m-%d').dt.date
results['FTR'] = np.where(results['FTR'] == 0, 'D', np.where(results['FTR'] == 1, 'H', 'A'))
results['HTHG'] = [int(re.search('(\d+)-(\d+)', values).group(1)) for values in results['scores_ht_score']]
results['HTAG'] = [int(re.search('(\d+)-(\d+)', values).group(2)) for values in results['scores_ht_score']]
results['HTR'] = np.where(results['HTHG'] > results['HTAG'], 'H', np.where(results['HTHG'] == results['HTAG'], 'D', 'A'))
results.drop(['league_name', 'season_name', 'scores_ht_score'], axis=1, inplace=True)
results['H'] = results['HomeTeam']
results['A'] = results['AwayTeam']
cols_to_keep = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'id', 'round_name', 'season_id']
appended_data_HOME = []
appended_data_AWAY = []

for i in tqdm(results['season_id'].unique()):
    for j in results['round_name'].unique():
        resultG = results[(results['season_id'] == i) & (results['round_name'] <= j)]
        team_results = pd.melt(resultG, id_vars=cols_to_keep, value_vars=['H', 'A'], var_name='Home/Away', value_name='Team')
        team_results['Opponent'] = np.where(team_results['Team'] == team_results['HomeTeam'], team_results['AwayTeam'], team_results['HomeTeam'])
        points_map = {'W': 3,'D': 1,'L': 0}
        # full time goals
        team_results['Goals'] = np.where(team_results['Team'] == team_results['HomeTeam'],team_results['FTHG'],team_results['FTAG'])
        team_results['Goals_Opp'] = np.where(team_results['Team'] != team_results['HomeTeam'],team_results['FTHG'],team_results['FTAG'])
        team_results['Result'] = np.vectorize(get_result)(team_results['Goals'], team_results['Goals_Opp'])
        team_results['Points'] = team_results['Result'].map(points_map)
        # 1st half goals
        team_results['1H_Goals'] = np.where(team_results['Team'] == team_results['HomeTeam'],team_results['HTHG'],team_results['HTAG'])
        team_results['1H_Goals_Opp'] = np.where(team_results['Team'] != team_results['HomeTeam'],team_results['HTHG'],team_results['HTAG'])
        team_results['1H_Result'] = np.vectorize(get_result)(team_results['1H_Goals'], team_results['1H_Goals_Opp'])
        team_results['1H_Points'] = team_results['1H_Result'].map(points_map)
        # 2nd half goals
        team_results['2H_Goals'] = team_results['Goals'] - team_results['1H_Goals']
        team_results['2H_Goals_Opp'] = team_results['Goals_Opp'] - team_results['1H_Goals_Opp']
        team_results['2H_Result'] = np.vectorize(get_result)(team_results['2H_Goals'], team_results['2H_Goals_Opp'])
        team_results['2H_Points'] = team_results['2H_Result'].map(points_map)
        # Drop unnecessary columns and sort by date
        cols_to_drop = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
        team_results = (team_results.sort_values(by=['Date', 'id']))
        # Split for Home and Away
        team_results_Home = team_results[team_results['Home/Away'] == 'H']
        team_results_Away = team_results[team_results['Home/Away'] == 'A']
        #### HOME ####
        results_byteam_HOME = team_results_Home.groupby(['Team'])
        # Rank Teams in Standings
        table_FT_H = (results_byteam_HOME.apply(standings,result_col='Result',goals_col='Goals',goals_opp_col='Goals_Opp',points_col='Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False))
        table_FT_H['rank'] = (table_FT_H.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int))
        table_FT_H['Type'] = 'FT'
        # Get League Table for First Half Goals only
        table_1H_H = (results_byteam_HOME.apply(standings,result_col='1H_Result',goals_col='1H_Goals',goals_opp_col='1H_Goals_Opp',points_col='1H_Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False))
        table_1H_H['rank'] = (table_1H_H.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int))
        table_1H_H['Type'] = '1H'
        # Get League Table_2H_H for First Half Goals only
        table_2H_H = (results_byteam_HOME.apply(standings,result_col='2H_Result',goals_col='2H_Goals',goals_opp_col='2H_Goals_Opp',points_col='2H_Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False))
        table_2H_H['rank'] = (table_2H_H.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int))
        table_2H_H['Type'] = '2H'
        table_merged_H = pd.concat([table_FT_H,table_1H_H, table_2H_H])
        table_merged_H['round_name'] = j
        table_merged_H['season_id'] = i
        appended_data_HOME.append(table_merged_H)
        #### AWAY ####
        results_byteam_AWAY = team_results_Away.groupby(['Team'])
        # Rank Teams in Standings
        table_FT_A = (results_byteam_AWAY.apply(standings,result_col='Result',goals_col='Goals',goals_opp_col='Goals_Opp',points_col='Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False))
        table_FT_A['rank'] = (table_FT_A.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int))
        table_FT_A['Type'] = 'FT'
        # Get League Table for First Half Goals only
        table_1H_A = (results_byteam_AWAY.apply(standings,result_col='1H_Result',goals_col='1H_Goals',goals_opp_col='1H_Goals_Opp',points_col='1H_Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False))
        table_1H_A['rank'] = (table_1H_A.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int))
        table_1H_A['Type'] = '1H'
        # Get League Table_2H_A for First Half Goals only
        table_2H_A = (results_byteam_AWAY.apply(standings,result_col='2H_Result',goals_col='2H_Goals',goals_opp_col='2H_Goals_Opp',points_col='2H_Points').sort_values(by=['Points', 'GD', 'GF'], ascending=False))
        table_2H_A['rank'] = (table_2H_A.apply(lambda row: (row['Points'], row['GD'], row['GF']), axis=1).rank(method='min', ascending=False).astype(int))
        table_2H_A['Type'] = '2H'
        table_merged_A = pd.concat([table_FT_A,table_1H_A, table_2H_A])
        table_merged_A['round_name'] = j
        table_merged_A['season_id'] = i
        appended_data_AWAY.append(table_merged_A)

appended_data_HOME = pd.concat(appended_data_HOME)
appended_data_AWAY = pd.concat(appended_data_AWAY)

  0%|          | 0/30 [00:00<?, ?it/s]

#### Checks and Export

In [8]:
print(len(appended_data_HOME))
appended_data_HOME.head(300)

66051


,Played,Won,Drawn,Lost,GF,GA,GD,Points,rank,Type,round_name,season_id
Team,,,,,,,,,,,,
Bordeaux,1,1,0,0,3,2,1,3,1,FT,1,765
Caen,1,1,0,0,3,2,1,3,1,FT,1,765
Metz,1,1,0,0,3,2,1,3,1,FT,1,765
Montpellier,1,1,0,0,1,0,1,3,4,FT,1,765
Nice,1,1,0,0,1,0,1,3,4,FT,1,765
...,...,...,...,...,...,...,...,...,...,...,...,...
Paris Saint Germain,3,1,2,0,2,0,2,5,6,1H,6,765
Guingamp,3,1,2,0,2,1,1,5,7,1H,6,765
Rennes,3,1,2,0,2,1,1,5,7,1H,6,765


In [9]:
print(len(appended_data_AWAY))
appended_data_AWAY.head(300)

66078


,Played,Won,Drawn,Lost,GF,GA,GD,Points,rank,Type,round_name,season_id
Team,,,,,,,,,,,,
Olympique Lyonnais,1,1,0,0,3,0,3,3,1,FT,1,765
Nantes,1,1,0,0,1,0,1,3,2,FT,1,765
Paris Saint Germain,1,1,0,0,1,0,1,3,2,FT,1,765
Guingamp,1,0,1,0,2,2,0,1,4,FT,1,765
Toulouse,1,0,1,0,0,0,0,1,5,FT,1,765
...,...,...,...,...,...,...,...,...,...,...,...,...
Lille,3,1,2,0,2,1,1,5,6,1H,6,765
Paris Saint Germain,3,1,1,1,4,2,2,4,7,1H,6,765
Nice,2,1,1,0,1,0,1,4,8,1H,6,765


In [ ]:
appended_data_HOME.to_csv('Output/tables_HOME.csv')
appended_data_AWAY.to_csv('Output/tables_AWAY.csv')